In [8]:
import pandas as pd
import zipfile
from sklearn.impute import MissingIndicator, SimpleImputer
import numpy as np
from fancyimpute import IterativeImputer, knn

In [9]:
# Small_df
df_small_train = pd.read_csv(zipfile.ZipFile('../../dist/orange_small_train.data.zip').open('orange_small_train.data'),sep='\t')
df_small_test = pd.read_csv(zipfile.ZipFile('../../dist/orange_small_test.data.zip').open('orange_small_test.data'),sep='\t')

# labels
df_small_train['appetency'] = pd.read_csv('../../dist/orange_small_train_appetency.labels',header = None)
df_small_train['churn'] = pd.read_csv('../../dist/orange_small_train_churn.labels',header = None)
df_small_train['upselling'] = pd.read_csv('../../dist/orange_small_train_upselling.labels',header = None)

In [10]:
df = df_small_train.copy()
df2 = df[[column for column in df if df[column].count() / len(df) >= 0.3]]
print("List of dropped columns:", end=" ")
for c in df.columns:
    if c not in df2.columns:
        print(c, end=", ")
print('\n')
df = df2

List of dropped columns: Var1, Var2, Var3, Var4, Var5, Var8, Var9, Var10, Var11, Var12, Var14, Var15, Var16, Var17, Var18, Var19, Var20, Var23, Var26, Var27, Var29, Var30, Var31, Var32, Var33, Var34, Var36, Var37, Var39, Var40, Var41, Var42, Var43, Var45, Var46, Var47, Var48, Var49, Var50, Var51, Var52, Var53, Var54, Var55, Var56, Var58, Var59, Var60, Var61, Var62, Var63, Var64, Var66, Var67, Var68, Var69, Var70, Var71, Var75, Var77, Var79, Var80, Var82, Var84, Var86, Var87, Var88, Var89, Var90, Var91, Var92, Var93, Var95, Var96, Var97, Var98, Var99, Var100, Var101, Var102, Var103, Var104, Var105, Var106, Var107, Var108, Var110, Var111, Var114, Var115, Var116, Var117, Var118, Var120, Var121, Var122, Var124, Var127, Var128, Var129, Var130, Var131, Var135, Var136, Var137, Var138, Var139, Var141, Var142, Var145, Var146, Var147, Var148, Var150, Var151, Var152, Var154, Var155, Var156, Var157, Var158, Var159, Var161, Var162, Var164, Var165, Var166, Var167, Var168, Var169, Var170, Var171, Var

In [11]:
df_v2 = df[:1000].copy()
# df_v2 = pd.get_dummies(df_v2,dummy_na=True,)+

for cat_feature in df_v2.select_dtypes(include=['object']).columns:
    df_v2[cat_feature] = pd.Categorical(df_v2[cat_feature]).codes

df_v2.head()

,Var6,Var7,Var13,Var21,Var22,Var24,Var25,Var28,Var35,Var38,...,Var222,Var223,Var225,Var226,Var227,Var228,Var229,appetency,churn,upselling
0,1526.0,7.0,184.0,464.0,580.0,14.0,128.0,166.56,0.0,3570.0,...,404,3,-1,22,2,5,-1,-1,-1,-1
1,525.0,0.0,0.0,168.0,210.0,2.0,24.0,353.52,0.0,4764966.0,...,20,0,-1,14,2,5,-1,-1,1,-1
2,5236.0,7.0,904.0,1212.0,1515.0,26.0,816.0,220.08,0.0,5883894.0,...,214,3,1,10,0,18,0,-1,-1,-1
3,NaN,0.0,0.0,NaN,0.0,NaN,0.0,22.08,0.0,0.0,...,109,0,-1,7,2,5,-1,-1,-1,-1
4,1029.0,7.0,3216.0,64.0,80.0,4.0,64.0,200.00,0.0,0.0,...,13,0,1,7,2,5,1,-1,-1,-1


In [12]:
imp_cols = df_v2.columns.values
imp_cols

array(['Var6', 'Var7', 'Var13', 'Var21', 'Var22', 'Var24', 'Var25',
       'Var28', 'Var35', 'Var38', 'Var44', 'Var57', 'Var65', 'Var72',
       'Var73', 'Var74', 'Var76', 'Var78', 'Var81', 'Var83', 'Var85',
       'Var94', 'Var109', 'Var112', 'Var113', 'Var119', 'Var123',
       'Var125', 'Var126', 'Var132', 'Var133', 'Var134', 'Var140',
       'Var143', 'Var144', 'Var149', 'Var153', 'Var160', 'Var163',
       'Var173', 'Var181', 'Var189', 'Var192', 'Var193', 'Var195',
       'Var196', 'Var197', 'Var198', 'Var199', 'Var200', 'Var202',
       'Var203', 'Var204', 'Var205', 'Var206', 'Var207', 'Var208',
       'Var210', 'Var211', 'Var212', 'Var214', 'Var216', 'Var217',
       'Var218', 'Var219', 'Var220', 'Var221', 'Var222', 'Var223',
       'Var225', 'Var226', 'Var227', 'Var228', 'Var229', 'appetency',
       'churn', 'upselling'], dtype=object)

In [13]:
%%time
df_train_imputed = pd.DataFrame(IterativeImputer(verbose=True).fit_transform(df_v2),columns= imp_cols)

# df_train_imputed['Loan_Status'] = df_train['Loan_Status']
# df_train_imputed.Loan_Status = df_train_imputed.Loan_Status.map(dict(Y=1,N=0))
# df_train_imputed.head()

[IterativeImputer] Completing matrix with shape (1000, 77)
[IterativeImputer] Ending imputation round 1/10, elapsed time 0.25


C:\Users\mdabati\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:993: RuntimeWarning: divide by zero encountered in true_divide
  return (c / G_diag) ** 2, c
C:\Users\mdabati\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:971: RuntimeWarning: overflow encountered in square
  v = s ** 2
C:\Users\mdabati\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\base.py:198: RuntimeWarning: invalid value encountered in add
  dense_output=True) + self.intercept_
C:\Users\mdabati\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:971: RuntimeWarning: overflow encountered in square
  v = s ** 2


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [14]:
df_train_imputed.head()

NameError: name 'df_train_imputed' is not defined

In [ ]:
df_v2.head()

In [ ]:
s = pd.Series(list('abca'))

pd.get_dummies(s)